### Setup imports

In [10]:
!uv pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124

Using Python 3.12.8 environment at: C:\Users\jesse\Documents\TU Delft\MSE git\img_classification\.venv
Audited 2 packages in 6ms


In [11]:
import os
import time
import torch
import numpy
from torchvision import datasets, transforms, models
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

print(torch.cuda.is_available())

True


### Load datasets

In [12]:
DATASET_PATH = os.path.join(os.getcwd(), "../training_data/EuroSAT_RGB")

BATCH_SIZE = 1500
TEST_SIZE = 0.2
EPOCHS = 10


In [13]:
transform = transforms.Compose([
  transforms.ToTensor()
])

In [14]:
dataset_full = datasets.ImageFolder(root=DATASET_PATH, transform=transform)

# Split into training and validation set
train_idx, val_idx = train_test_split(
  range(len(dataset_full)),       # Size of the full dataset
  test_size=TEST_SIZE,            # Fraction used for validation (test)
  stratify=dataset_full.targets   # Ensure class distribution is preserved
)

# Create the training and validation set
dataset_train = Subset(dataset_full, train_idx)
dataset_val = Subset(dataset_full, val_idx)

# TODO: Investigate if the DataLoader should shuffle the data
loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=False)
loader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False)

# Show all the class labels
# print(dataset_full.classes)

### Set Model

In [15]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
# Adjust to the amount of classes
model.fc = torch.nn.Linear(model.fc.in_features, len(dataset_full.classes))

### Setup Training

In [16]:
# Set up loss function
loss = torch.nn.CrossEntropyLoss()

# Set up optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

# Check if a GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cuda


In [17]:
skip_training = False

### Model Training

In [18]:
if not(skip_training):

  # Track time
  start_time = time.time()

  # Holder for training loss
  train_loss = []
  # Holder for validation loss
  validation_loss = []

  for epoch in tqdm(range(EPOCHS), desc='Epochs'):

    # Setup model to perform training
    model.train()

    loss_train = 0
    # Number of correct predictions for train split
    train_correct = 0
    # Number of total predictions for train split
    train_total = 0

    # Perform training in batches
    for inputs, labels in tqdm(loader_train, desc="Training - batch"):
      # Move data to device (CPU or GPU)
      inputs, labels = inputs.to(device), labels.to(device)

      # Zero the gradients
      optimizer.zero_grad()
      # Calculate the predictions (outputs) on the batch
      outputs = model(inputs)
      # Calculate the loss (error) between predictions and labels
      train_loss = loss(outputs, labels)
      # Back propagate the error and update the weights
      train_loss.backward()
      # Update the weights using the SGD optimizer
      optimizer.step()
      # Extract the predicted class
      predicted  torch.max(outputs, 1)[1]
      # Update the total and number of correct
      train_correct += (predicted == labels).sum().item()
      train_total += labels.size(0) 

    # train_acc = 100. * correct / total

    # Setup model to perform validation (inference)
    model.eval()

    # Number of correct predictions for validation split
    val_correct = 0
    # Number of total predictions for validation split
    val_total = 0

    # Validate model using the validation set, the gradients do not need to be calculated
    with torch.no_grad():
      for inputs, labels in tqdm(loader_val, desc="Validation - batch"):
        # Move data to device (CPU or GPU)
        inputs, labels = inputs.to(device), labels.to(device)

        # Calculate the predictions (outputs) on the batch
        outputs = model(inputs)
        # Calculate the loss (error) between predictions and labels
        val_loss = loss(outputs, labels)

  # Saving the model
  torch.save(model.state_dict(), "model.pth")
  print("Model trained")
else:
  print("Skip training...")
  try:
    model.load_state_dict(torch.load("model.pth"))
    print("Model loaded")
  except FileNotFoundError:
    print("Model not found")

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Training - batch:   0%|          | 0/15 [00:00<?, ?it/s]

Validation - batch:   0%|          | 0/4 [00:00<?, ?it/s]

Training - batch:   0%|          | 0/15 [00:00<?, ?it/s]

Validation - batch:   0%|          | 0/4 [00:00<?, ?it/s]

...

Training - batch:   0%|          | 0/15 [00:00<?, ?it/s]

Validation - batch:   0%|          | 0/4 [00:00<?, ?it/s]

Training - batch:   0%|          | 0/15 [00:00<?, ?it/s]

Validation - batch:   0%|          | 0/4 [00:00<?, ?it/s]

Model trained
Elapsed time: 117.93 seconds


### Plot losses over training epochs

if torch.is_tensor():